In [1]:
%pylab
import numpy as np
import pandas as pd

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
path = r'E:\Work\06-Work\Data Anlysis\Profit Loss\Mon8B_PA\Mon8_product.xls'
sheet_name = 'Single'
cols = ['erp_id','order_id','img_url','sku','sale_num',
        'carrier','channel','track_number','weight',
       'weight_cal','get_money','purchase','freight',
       'package_fee','profit','paid_time','store','ship_time',
       'country'
       ]
df = pd.read_excel(path, sheet_name, headers=1)
df.columns = cols

In [3]:
df.head()

,erp_id,order_id,img_url,sku,sale_num,carrier,channel,track_number,weight,weight_cal,get_money,purchase,freight,package_fee,profit,paid_time,store,ship_time,country
0,330270,109-7653211-3115408,http://ecx.images-amazon.com/images/I/410GS9o9...,BKQK128black,1,邮政速递,【邮政速递】E邮宝,LT091700416CN,485.0,485.0,69.88,45.17,45.38,0.5,-21.17,2016-08-19 05:43:47,Amazon NH,2016-08-20 15:23:13,United States
1,330697,110-5272201-2640228,http://ecx.images-amazon.com/images/I/41hUfg1s...,B9210Lshape,3,邮政速递,【邮政速递】E邮宝,LT099016410CN,220.0,220.0,44.66,9.57,25.50,0.5,9.09,2016-08-21 04:33:31,Amazon NH,2016-08-22 14:33:12,United States
2,330191,111-1937499-6016207,http://ecx.images-amazon.com/images/I/410GS9o9...,BKQK128black,1,邮政速递,【邮政速递】E邮宝,LT091700362CN,476.0,476.0,69.88,45.17,44.70,0.5,-20.49,2016-08-19 00:00:35,Amazon NH,2016-08-20 15:23:01,United States
3,329932,113-3712141-2969826,http://ecx.images-amazon.com/images/I/41hUfg1s...,B9210Lshape,3,邮政速递,【邮政速递】E邮宝,LT087717579CN,219.0,219.0,44.66,9.57,25.43,0.5,9.16,2016-08-17 23:57:53,Amazon NH,2016-08-18 15:45:05,United States
4,329589,113-5556257-3369018,http://ecx.images-amazon.com/images/I/51nDqJQV...,BGRB059blueL,1,邮政速递,【邮政速递】E邮宝,LT082031506CN,73.0,73.0,38.09,21.53,14.84,0.5,1.22,2016-08-16 08:09:32,Amazon NH,2016-08-16 13:50:26,United States


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2876 entries, 0 to 2875
Data columns (total 19 columns):
erp_id          2876 non-null int64
order_id        2876 non-null object
img_url         2876 non-null object
sku             2876 non-null object
sale_num        2876 non-null int64
carrier         2867 non-null object
channel         2876 non-null object
track_number    2872 non-null object
weight          2744 non-null float64
weight_cal      2657 non-null float64
get_money       2876 non-null float64
purchase        2876 non-null float64
freight         2794 non-null float64
package_fee     2874 non-null float64
profit          2876 non-null float64
paid_time       2876 non-null datetime64[ns]
store           2876 non-null object
ship_time       2876 non-null datetime64[ns]
country         2876 non-null object
dtypes: datetime64[ns](2), float64(7), int64(2), object(8)
memory usage: 337.1+ KB


#### 分析问题： 做盈亏分析，从数据里分析出盈利款TOP10,亏损款TOP10
1. 把SKU,销量，利润分别列出来
2. 对SKU进行分析，得到SPU
3. 按SPU分组汇总

In [5]:
profit_loss = df.loc[:,['sku','sale_num','profit']]
profit_loss.head()

,sku,sale_num,profit
0,BKQK128black,1,-21.17
1,B9210Lshape,3,9.09
2,BKQK128black,1,-20.49
3,B9210Lshape,3,9.16
4,BGRB059blueL,1,1.22


In [6]:
# 定义函数，获得SPU
def get_spu(item):
    if item.startswith('B9'):
        spu,attr = item[:5],item[5:]
    elif item.startswith('EATGE'):
        spu,attr = item[:11],item[12:]
    elif item.startswith('WZ'):
        spu,attr = item[:9],item[9:]
    else:
        spu,attr = item[:7],item[7:]
        if attr.startswith('-'):
            attr = attr[1:]
    return spu,attr

In [12]:
profit_loss = df.loc[:,['sku','sale_num','profit']]
spu = pd.Series([i[0] for i in profit_loss.sku.map(get_spu)])
attr = pd.Series([i[-1] for i in profit_loss.sku.map(get_spu)])
profit_loss['SPU'],profit_loss['attr'] = spu,attr

In [13]:
spu_profit = profit_loss.groupby('SPU').sum().sort_values(by='profit')
spu_profit.head()

,sale_num,profit
SPU,,
BUCJ402,10,-81.82
BKQK128,2,-41.66
STXR101,4,-23.98
EATGE001820,1,-23.58
EATGE001818,1,-17.35


In [14]:
spu_profit['avg_profit'] = round(spu_profit.profit / spu_profit.sale_num,2)

In [41]:
spu_profit[spu_profit['avg_profit'] < 0]

,sale_num,profit,avg_profit
spu,,,
BUCJ402,10,-81.8200,-8.18
BKQK128,2,-41.6600,-20.83
STXR101,4,-23.9800,-6.00
EATGE001820,1,-23.5800,-23.58
EATGE001818,1,-17.3500,-17.35
KCXC459,1,-13.7300,-13.73
BHBC030,1,-3.7000,-3.70
CSOO201,91,-3.5427,-0.04
DHXX005,1,-3.3100,-3.31


In [15]:
path = r'E:\Work\06-Work\Data Anlysis\Profit Loss\Mon8B_PA'
excel_name = 'profit.xlsx'
sheet_name = ['loss','profit','total']
cols = ['销量','利润','单个利润']
spu_profit.columns = cols
import os

In [53]:
full_path = os.path.join(path,excel_name)
with pd.ExcelWriter(full_path) as writer:
    spu_profit.head(10).to_excel(writer,sheet_name[0])
    spu_profit.tail(10).to_excel(writer,sheet_name[1])
    spu_profit.to_excel(writer,sheet_name[-1])

In [16]:
import matplotlib  
import matplotlib.pyplot as plt 
plt.scatter(spu_profit['销量'],spu_profit['利润']) 

In [20]:
country_df = df.groupby('country').sum().sort_values(by='profit')
country_df.head(10)

,erp_id,sale_num,weight,weight_cal,get_money,purchase,freight,package_fee,profit
country,,,,,,,,,
South Africa,1662072,8,1574.0,401.0,242.55,105.28,209.17,6.6,-78.50
Cambodia,331846,1,169.0,169.0,34.03,18.50,21.78,1.3,-7.55
Martinique,332539,1,254.0,254.0,79.24,34.43,40.58,1.3,2.93
Seychelles,332168,1,97.0,97.0,41.14,17.50,19.31,1.3,3.03
Guadeloupe,332978,1,93.0,93.0,39.95,16.21,18.76,1.3,3.68
New Caledonia,332720,1,95.0,95.0,41.62,17.50,19.03,1.3,3.79
Tunisia,331293,1,92.0,92.0,39.95,15.91,18.63,1.3,4.11
Cyprus,664058,2,152.0,152.0,62.61,32.03,22.91,2.6,5.07
Costa Rica,663466,2,132.0,132.0,60.52,20.84,30.21,2.6,6.87


In [22]:
# 按快递分析
carrier_df = df.groupby('carrier').sum().sort_values(by='profit')
carrier_df

,erp_id,sale_num,weight,weight_cal,get_money,purchase,freight,package_fee,profit
carrier,,,,,,,,,
--请选择--,2991631,12,4644.0,NaN,1124.42,528.16,152.110,11.000,433.150
邮政速递,60322101,208,34150.0,31627.0,9517.00,3858.19,4098.620,89.500,1470.690
线上,33804579,118,16500.7,12997.0,5161.01,1566.72,1264.160,128.700,2344.210
平 川,853319134,2802,609524.5,601276.0,164058.44,68641.97,60248.128,3251.733,32037.609


In [24]:
# 按购买量分类
sale_num_df = df.groupby('sale_num').sum()
sale_num_df

,erp_id,weight,weight_cal,get_money,purchase,freight,package_fee,profit
sale_num,,,,,,,,
1,908670574,629423.8,609485.0,167963.47,70262.52,61831.7573,3341.0164,32649.1763
2,25857600,23873.7,23873.0,6429.54,2495.34,2458.0796,88.9333,1387.1871
3,11270610,6009.0,6009.0,1879.63,629.85,681.6762,39.6333,528.4705
4,995847,1472.0,299.0,250.14,123.04,180.5900,3.2000,-56.6900
5,1989376,3334.0,3334.0,1465.76,614.30,332.5582,5.0500,513.8518
6,2324123,1958.0,1958.0,567.02,162.36,186.7000,8.3000,209.6600
8,330123,175.0,175.0,34.21,8.88,20.9800,1.3000,3.0500
9,332064,391.0,391.0,129.23,36.09,42.2900,1.3000,49.5500
10,1324025,376.0,376.0,1408.29,425.10,38.0100,1.3000,943.8800


In [25]:
df.describe()

d:\program files\python\python35-32\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,erp_id,sale_num,weight,weight_cal,get_money,purchase,freight,package_fee,profit
count,2876.000000,2876.000000,2744.000000,2657.000000,2876.000000,2876.000000,2794.000000,2874.000000,2876.000000
mean,331511.100139,1.095619,243.080612,243.093715,62.802726,26.043206,23.540673,1.214347,12.768250
std,1056.719343,0.596426,244.746705,244.480610,51.101920,25.400601,17.923154,0.247116,19.280425
min,329589.000000,1.000000,0.000000,7.000000,11.780000,0.000000,0.000000,0.000000,-90.870000
25%,330607.750000,1.000000,NaN,NaN,34.980000,12.980000,NaN,NaN,3.687500
50%,331514.500000,1.000000,NaN,NaN,44.850000,17.500000,NaN,NaN,9.060000
75%,332420.250000,1.000000,NaN,NaN,68.650000,32.880000,NaN,NaN,15.455000
max,333669.000000,11.000000,1208.000000,1208.000000,581.730000,385.000000,176.720000,1.400000,493.350000


In [26]:
df.count()

erp_id          2876
order_id        2876
img_url         2876
sku             2876
sale_num        2876
carrier         2867
channel         2876
track_number    2872
weight          2744
weight_cal      2657
get_money       2876
purchase        2876
freight         2794
package_fee     2874
profit          2876
paid_time       2876
store           2876
ship_time       2876
country         2876
dtype: int64

In [28]:
# 按店铺分析
store_df = df.groupby('store').sum()
store_df

,erp_id,sale_num,weight,weight_cal,get_money,purchase,freight,package_fee,profit
store,,,,,,,,,
AG,141254689,459,33419.0,32077.0,14232.10,6613.21,5371.6699,509.3499,1737.8702
Amazon NH,1650679,9,1473.0,1473.0,267.17,131.01,155.8500,2.5000,-22.1900
ET,193038592,628,95235.7,89896.0,28701.17,8151.44,10351.5717,686.9498,9653.9885
EU,121325818,464,65460.0,63969.0,19696.69,9053.68,7121.0899,435.4999,3086.4202
HE,41771014,129,22397.4,21308.0,6150.30,1815.64,2451.0700,146.4500,1737.1400
NH,447771850,1441,445302.1,433451.0,110781.28,48769.14,39889.8998,1692.5834,20550.6568
Wish,993231,3,659.0,659.0,132.60,52.44,64.1500,3.1000,12.9100
babysafety2015,5288604,16,2751.0,2751.0,513.68,260.70,339.1600,12.3000,-98.4800
线下店,331447,2,316.0,316.0,145.65,53.00,28.1800,1.3000,63.1700


In [31]:
df.dtypes

erp_id                   int64
order_id                object
img_url                 object
sku                     object
sale_num                 int64
carrier                 object
channel                 object
track_number            object
weight                 float64
weight_cal             float64
get_money              float64
purchase               float64
freight                float64
package_fee            float64
profit                 float64
paid_time       datetime64[ns]
store                   object
ship_time       datetime64[ns]
country                 object
dtype: object